# Keras MLP

In [35]:
import pandas as pd
import numpy as np
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import SGD

In [36]:
df = pd.read_excel("Dry_Bean_Dataset.xlsx")
df = df.drop_duplicates()
X = df.iloc[:,:16]
y = df.iloc[:,16:]
y = y.reset_index().drop(columns = "index")
scaler = StandardScaler()
X = pd.DataFrame(scaler.fit_transform(X), columns = X.columns)
pt = PowerTransformer(method = "yeo-johnson")
X = pd.DataFrame(pt.fit_transform(X), columns = X.columns)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 101)
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.fit_transform(y_test)
y_train = tf.keras.utils.to_categorical(y_train, num_classes=7)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=7)

C:\Users\matth\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


In [55]:
model = tf.keras.Sequential()
model.add(layers.Dense(16, input_dim = X_train.shape[1], activation = "relu")) # input layer requires input_dim param
model.add(layers.Dense(32, activation = "tanh"))
model.add(layers.Dense(64, activation = "tanh"))
model.add(layers.Dense(32, activation = "tanh"))
model.add(layers.Dense(16, activation = "tanh"))
model.add(layers.Dense(7, activation='softmax'))
model.summary()
model.compile(loss="kullback_leibler_divergence", optimizer= "adam", metrics=['accuracy'])
es = tf.keras.callbacks.EarlyStopping(monitor='loss', min_delta=0.00005, patience=3, verbose=1, mode='auto')
with tf.device('/cpu:0'):
    model.fit(X_train, y_train, epochs = 100, shuffle = True, batch_size=32, verbose=1, callbacks=[es])
    score = model.evaluate(X_test, y_test, verbose=0)

Model: "sequential_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_82 (Dense)            (None, 16)                272       
                                                                 
 dense_83 (Dense)            (None, 32)                544       
                                                                 
 dense_84 (Dense)            (None, 64)                2112      
                                                                 
 dense_85 (Dense)            (None, 32)                2080      
                                                                 
 dense_86 (Dense)            (None, 16)                528       
                                                                 
 dense_87 (Dense)            (None, 7)                 119       
                                                                 
Total params: 5,655
Trainable params: 5,655
Non-train

In [56]:
predictions = model.predict(X_test)
y_pred=np.argmax(predictions, axis=1)
tests=np.argmax(y_test, axis=1)
print(confusion_matrix(tests, y_pred))
print(classification_report(tests,y_pred,target_names=y.Class.unique()))

85/85 [==============================] - 0s 2ms/step
[[246   0   8   0   0   2   4]
 [  0 115   0   0   0   0   0]
 [  9   1 324   0   1   1   3]
 [  0   0   0 649   0   8  56]
 [  0   0   5   3 332   0   8]
 [  1   0   0  11   0 388   9]
 [  1   0   3  30   5   7 479]]
              precision    recall  f1-score   support

       SEKER       0.96      0.95      0.95       260
    BARBUNYA       0.99      1.00      1.00       115
      BOMBAY       0.95      0.96      0.95       339
        CALI       0.94      0.91      0.92       713
       HOROZ       0.98      0.95      0.97       348
        SIRA       0.96      0.95      0.95       409
    DERMASON       0.86      0.91      0.88       525

    accuracy                           0.94      2709
   macro avg       0.95      0.95      0.95      2709
weighted avg       0.94      0.94      0.94      2709



In [57]:
metrics.accuracy_score(tests, y_pred)

0.935031376891842